In [ ]:
import tensorflow.keras as keras
keras.__version__

## Generator

In [ ]:
#import keras
from tensorflow.keras import layers
import numpy as np

latent_dim = 64
height = 64
width = 64
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

# First, transform the input into a 16x16 128-channels feature map
x = layers.Dense(3 * 32 * 32)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((32, 32, 3))(x)

# Then, add a convolution layer
x = layers.Conv2D(64, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Upsample to 32x32
x = layers.Conv2DTranspose(128, 2, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

# Few more conv layers
x = layers.Conv2D(128, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Produce a 32x32 1-channel feature map
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

## Discriminator

In [ ]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, padding = 'same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(64, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(64, 4, padding = 'same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(64, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(32, 4, padding = 'same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(32, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)

# One dropout layer - important trick!
x = layers.Dropout(0.4)(x)

# Classification layer
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

# To stabilize training, we use learning rate decay
# and gradient clipping (by value) in the optimizer.
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

## Model

In [ ]:
# Set discriminator weights to non-trainable
# (will only apply to the `gan` model)
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

## GAN 모델 불러오기

In [ ]:
gan.load_weights('gan_tune_2_1_4300.h5')

In [ ]:
#discriminator.trainable=True
discriminator.summary()

In [ ]:
gan.summary()

In [ ]:
import os
from keras.preprocessing import image
from PIL import Image
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

# # Load CIFAR10 data
# (x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

## Image Upload

In [ ]:
filename_list = os.listdir('C:/Users/onest/OneDrive/바탕 화면/Python/jolie_project/Angelina Jolie_Insta')

img_list = []
num = 0
err_num = 0
for file in filename_list:
    try:
        img = Image.open('C:/Users/onest/OneDrive/바탕 화면/Python/jolie_project/Angelina Jolie_Insta/%s'%file)
    except:
        print(file)
        err_num += 1
        continue
    pixels = load_img('C:/Users/onest/OneDrive/바탕 화면/Python/jolie_project/Angelina Jolie_Insta/%s'%file, target_size=(64,64))
    pixels = img_to_array(pixels)
    img_list.append(pixels)
img_arr = np.asarray(img_list)
img_arr = img_arr.astype('float32')/255.

In [ ]:
x_train = img_arr
x_train.shape

## Train Model

In [ ]:
%%time

# Select frog images (class 6)
# x_train = x_train[y_train.flatten() == 6]

# Normalize data
# x_train = x_train.reshape(
#     (x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = 'gan_tune_3/' # change

# Start training loop
start = 0
for step in range(iterations):
    # Sample random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    # print(len(random_latent_vectors))
    # Decode them to fake images
    generated_images = generator.predict(random_latent_vectors)
    generated_images = (generated_images+1)/2.
    # Combine them with real images
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # Assemble labels discriminating real from fake images
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # Add random noise to the labels - important trick!
    labels += 0.05 * np.random.random(labels.shape)

    # Train the discriminator
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # sample random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Assemble labels that say "all real images"
    misleading_targets = np.zeros((batch_size, 1))

    # Train the generator (via the gan model,
    # where the discriminator weights are frozen)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0

    # Occasionally save / plot
    if step % 50 == 0:
        # Save model weights
        gan.save_weights('gan_tune_2_2_' + str(step) + '.h5') # change

        # Print metrics
        print('discriminator loss at step %s: %s' % (step, d_loss))
        print('adversarial loss at step %s: %s' % (step, a_loss))

        # Save one generated image
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_jolie' + str(step) + '.png'))

        # Save one real image, for comparison
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_jolie' + str(step) + '.png'))

In [ ]:
generated_images.shape

In [ ]:
real_images.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Sample random points in the latent space
random_latent_vectors = np.random.normal(size=(10, latent_dim))

# Decode them to fake images
generated_images = generator.predict(random_latent_vectors)

for i in range(generated_images.shape[0]):
    img = image.array_to_img(generated_images[i] * 255., scale=False)
#    img = image.array_to_img(generated_images[i], scale=True)
    plt.figure()
    plt.imshow(img)
    
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Sample random points in the latent space
random_latent_vectors = np.random.normal(size=(20, 128))

# Decode them to fake images
generated_images = generator.predict(random_latent_vectors)

for i in range(generated_images.shape[0]):
    img = image.array_to_img(generated_images[i] * 255., scale=False)
#    img = image.array_to_img(generated_images[i], scale=True)
    plt.figure()
    plt.imshow(img)
    
plt.show()